In [ ]:
import numpy as np
import pandas as pd
import sys
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from textblob import TextBlob
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm

In [ ]:
books = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATASET/Books.csv',low_memory=False)
users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATASET/Users.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATASET/Ratings.csv')

In [ ]:
#Show first five rows of books

books.head()


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
users.head()


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
print(books.columns)
print(users.columns)
print(ratings.columns)


Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Index(['User-ID', 'Location', 'Age'], dtype='object')
Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


In [ ]:
data_books = books
data_users = users
data_ratings = ratings
# Replace 'your_books_dataset.csv', 'your_users_dataset.csv', 'your_ratings_dataset.csv'
# with the correct file paths or filenames of your datasets

data_books['Year-Of-Publication'].fillna(0, inplace=True)
data_books['Publisher'].fillna('Unknown', inplace=True)
# Replace null values in 'Year-Of-Publication' column with 0
# Replace null values in 'Publisher' column with 'Unknown'

# Replace null values in the users dataset
data_users['Age'].fillna(data_users['Age'].mean(), inplace=True)
# Replace null values in the 'Age' column with the mean value of the column

# Replace null values in the ratings dataset
data_ratings['User-ID'].fillna(0, inplace=True)
data_ratings['ISBN'].fillna('Unknown', inplace=True)
data_ratings['Book-Rating'].fillna(0, inplace=True)
# Replace null values in 'User-ID' column with 0
# Replace null values in 'ISBN' column with 'Unknown'
# Replace null values in 'Book-Rating' column with 0


In [ ]:
#merge rating and users

Users_with_Ratings = ratings.merge(users,on='User-ID')
Users_with_Ratings

,User-ID,ISBN,Book-Rating,Location,Age
0,276725,034545104X,0,"tyler, texas, usa",34.751434
1,276726,0155061224,5,"seattle, washington, usa",34.751434
2,276727,0446520802,0,"h, new south wales, australia",16.000000
3,276729,052165615X,3,"rijeka, n/a, croatia",16.000000
4,276729,0521795028,6,"rijeka, n/a, croatia",16.000000
...,...,...,...,...,...
1149775,276704,1563526298,9,"cedar park, texas, usa",34.751434
1149776,276706,0679447156,0,"quebec, quebec, canada",18.000000
1149777,276709,0515107662,10,"mannington, west virginia, usa",38.000000
1149778,276721,0590442449,10,"providence, rhode island, usa",14.000000


In [ ]:
print(Users_with_Ratings.columns)

Index(['User-ID', 'ISBN', 'Book-Rating', 'Location', 'Age'], dtype='object')


In [ ]:
#merge rating and book

BName_with_Ratings = ratings.merge(books,on='ISBN')
BName_with_Ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...


In [ ]:
print(BName_with_Ratings.columns)

Index(['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L'],
      dtype='object')


In [ ]:
#check how many null items are there
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [ ]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [ ]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [ ]:
#LINEAR REGRESSION

data = Users_with_Ratings
# Replace 'your_dataset.csv' with the correct file path or filename of your dataset

# Prepare the data
X = data['Age'].values.reshape(-1, 1)  # Independent variable (user age)
y = data['Book-Rating'].values  # Dependent variable (book rating)

# Create and fit the linear regression model
model = LinearRegression()
model.fit(X, y)

# Extract the regression coefficients and intercept
slope = model.coef_[0]
intercept = model.intercept_

correlation = data['Age'].corr(data['Book-Rating'])
# Print the regression coefficients
print("Regression Coefficient: ", slope)
print("Intercept: ", intercept)
print("Correlation between User Age and Book Rating:", correlation)

Regression Coefficient:  -0.008414717962658391
Intercept:  3.1746759163250524
Correlation between User Age and Book Rating: -0.026699497594569164


In [ ]:
#SENTIMENT ANALYSIS


# Read the dataset into a DataFrame
data = BName_with_Ratings

# Select the book titles
book_titles = data['Book-Title']

# Perform sentiment analysis on book titles
sentiments = []
for title in book_titles:
    blob = TextBlob(title)
    sentiment = blob.sentiment.polarity
    sentiments.append(sentiment)

# Add the sentiment scores to the DataFrame
data['Sentiment'] = sentiments

# Print the sentiment scores
print(data[['Book-Title', 'Sentiment']])

                                                Book-Title  Sentiment
0                                     Flesh Tones: A Novel   0.000000
1                                     Flesh Tones: A Novel   0.000000
2                                     Flesh Tones: A Novel   0.000000
3                                     Flesh Tones: A Novel   0.000000
4                                     Flesh Tones: A Novel   0.000000
...                                                    ...        ...
1031131                                    Mostly Harmless   0.500000
1031132                                        Gray Matter   0.000000
1031133  Triplet Trouble and the Class Trip (Triplet Tr...  -0.200000
1031134  A Desert of Pure Feeling (Vintage Contemporaries)   0.214286
1031135  Perplexing Lateral Thinking Puzzles: Scholasti...   0.000000

[1031136 rows x 2 columns]


In [ ]:
# Load the BName_with_Ratings dataset into a DataFrame
data = BName_with_Ratings

# Select the features you want to use for clustering
features = data[['User-ID', 'Book-Rating']]

# Perform feature scaling if necessary

# Specify the number of clusters
num_clusters = 3

# Create a KMeans object with the desired number of clusters
kmeans = KMeans(n_clusters=num_clusters)

# Fit the KMeans model to the data
kmeans.fit(features)

# Get the cluster labels for each data point
labels = kmeans.labels_

# Print the cluster labels
print("Cluster Labels:")
print(labels)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster Labels:
[1 0 0 ... 1 1 1]


In [ ]:
data = BName_with_Ratings
features = ['User-ID', 'ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication']
target = 'Book-Rating'

# Encode categorical variables
le = LabelEncoder()
for feature in features:
    if data[feature].dtype == 'object':
        data[feature] = le.fit_transform(data[feature])

# Split the data into training and testing sets
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Fit the decision tree model to the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.48806175689043196


# POPULARITY BASED

---



In [ ]:
#book title with ratings

df_Num_Rating =BName_with_Ratings.groupby('Book-Title').count()['Book-Rating'].reset_index()
df_Num_Rating.rename(columns={'Book-Rating': 'Num_Ratings'},inplace=True)
df_Num_Rating


,Book-Title,Num_Ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [ ]:
#book title with average rating

df_average_Rating=BName_with_Ratings.groupby('Book-Title').mean()['Book-Rating'].reset_index()
df_average_Rating.rename(columns={'Book-Rating': 'Average_Rating'},inplace=True)
df_average_Rating

<ipython-input-16-53295a724454>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_average_Rating=BName_with_Ratings.groupby('Book-Title').mean()['Book-Rating'].reset_index()


,Book-Title,Average_Rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [ ]:
#merge average rating and num rating
df_popularBooks = df_Num_Rating.merge(df_average_Rating,on='Book-Title')
df_popularBooks


,Book-Title,Num_Ratings,Average_Rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [ ]:
#books whose ratings are greater than equal to 250
#Sort
df_popularBooks=df_popularBooks[df_popularBooks['Num_Ratings']>=250].sort_values('Average_Rating',ascending=False)
df_popularBooks

,Book-Title,Num_Ratings,Average_Rating
80434,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80422,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80441,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80426,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
...,...,...,...
227701,Vinegar Hill (Oprah's Book Club (Paperback)),265,2.245283
233635,Whispers,286,2.199301
143377,Presumed Innocent,294,2.139456
94382,Isle of Dogs,288,2.000000


In [ ]:
#to remove multiple copies of same book
df_popularBooks.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Num_Ratings']]

,Book-Title,Book-Author,Num_Ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,428
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,387
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,278
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,347
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,556
...,...,...,...
716,Vinegar Hill (Oprah's Book Club (Paperback)),A. Manette Ansay,265
717,Whispers,BELVA PLAIN,286
727,Presumed Innocent,Scott Turow,294
733,Isle of Dogs,Patricia Cornwell,288


# COLLABORATIVE FILTERING

In [ ]:
x=BName_with_Ratings.groupby('User-ID').count()['Book-Rating']>200
Users_200Rating=x[x].index #User ID who rated more than 200 books

In [ ]:
Rating_Filtered=BName_with_Ratings[BName_with_Ratings['User-ID'].isin(Users_200Rating)]

In [ ]:
y=Rating_Filtered.groupby('Book-Title').count()['Book-Rating']>=50
Books_Famous = y[y].index
Books_Famous

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=706)

In [ ]:
Rating_Final=Rating_Filtered[Rating_Filtered['Book-Title'].isin(Books_Famous)]


In [ ]:
Rating_Final.drop_duplicates()

In [ ]:
print(Rating_Final.columns)


Index(['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L', 'Sentiment'],
      dtype='object')


In [ ]:
pivottable=Rating_Final.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [ ]:
correlation_matrix = pivottable.corr()

# Print the correlation matrix
print("Correlation Matrix:")
print(correlation_matrix)

In [ ]:
pivottable.fillna(0,inplace=True)
pivottable

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
similarity_scores=cosine_similarity(pivottable)


In [ ]:
similarity_scores.shape

(706, 706)

In [ ]:
#we will give book name and it will return suggestions
def recommend(book_name):
    #index fetch ( on which number the book is )
    index=np.where(pivottable.index==book_name)[0][0]

    #along with distance item index will be visible also
    items_similar=sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:6] #sorting on the basis of similarity score

    for i in items_similar:
        print(pivottable.index[i[0]])

In [ ]:
(list(enumerate(similarity_scores[4])))

In [ ]:
recommend('Isle of Dogs')

Southern Cross
Shadow Prey
Chosen Prey
The Clinic (Alex Delaware Novels (Paperback))
Easy Prey


In [ ]:
recommend('Harry Potter and the Prisoner of Azkaban (Book 3)')

Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
